In [ ]:
from google.colab import files

uploaded = files.upload()

Saving data.csv to data.csv


#Importing Required Libraries
The first step in any data analysis or machine learning project is to import the necessary libraries.

In [ ]:
# Required Libraries
import numpy as np
import pandas as pd
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.cluster import KMeans
from sklearn.neighbors import NearestNeighbors
from sklearn.metrics.pairwise import cosine_similarity
import ipywidgets as widgets
from IPython.display import display, clear_output


**Explanation:**
numpy: For numerical operations.

pandas: For data manipulation and analysis.

StandardScaler: To standardize features by removing the mean and scaling to unit variance.

PCA: For dimensionality reduction.

KMeans: A clustering method.

NearestNeighbors: For finding the nearest neighbors of a set of points.

cosine_similarity: To compute similarity between arrays or matrices.

ipywidgets & IPython.display: For creating interactive GUI elements in the Jupyter notebook.

In [ ]:
# 1-Load and Preprocess the Data
data = pd.read_csv('data.csv')
data = data.fillna(data.mean())

<ipython-input-3-ed25b94996b1>:3: FutureWarning: The default value of numeric_only in DataFrame.mean is deprecated. In a future version, it will default to False. In addition, specifying 'numeric_only=None' is deprecated. Select only valid columns or specify the value of numeric_only to silence this warning.
  data = data.fillna(data.mean())


**Explanation:**
pd.read_csv(): Reads a CSV file into a Pandas DataFrame.

fillna(): Fills NaN values in the DataFrame. Here, it fills NaN values with the mean of each column.

In [ ]:
# Taking a subset of the data for faster execution
subset_data = data.sample(n=min(10000, len(data)), random_state=42)


**Explanation:**
This code randomly selects a subset of the data for faster execution. The size of the subset is the minimum between 10,000 and the size of the dataset.

In [ ]:
# 2. Feature Selection for the subset
features = subset_data[['valence', 'energy', 'danceability', 'acousticness', 'instrumentalness', 'liveness', 'loudness', 'speechiness', 'tempo']]
scaler = StandardScaler()
scaled_features = scaler.fit_transform(features)


**Explanation:**
Selects a subset of features from the dataset that are relevant for mood classification.
Uses the StandardScaler to scale these features for better performance of machine learning algorithms.

In [ ]:
# 2.1 Apply PCA for Dimensionality Reduction
pca = PCA(n_components=5)
principal_components = pca.fit_transform(scaled_features)


**Explanation:**
PCA (Principal Component Analysis) is applied to reduce the dimensionality of the data. Here, the dimensionality is reduced to 5 principal components.

In [ ]:
# 3. Mood Classification with K-means for the subset
kmeans = KMeans(n_clusters=5, random_state=42)
subset_data['cluster'] = kmeans.fit_predict(principal_components)

#Assigning Moods to Clusters
def assign_mood(cluster):
    if cluster == 0: return "Energetic"
    if cluster == 1: return "Relaxing"
    if cluster == 2: return "Sad"
    if cluster == 3: return "Intense"  # Replaced "Aggressive" with "Intense"
    return "Happy"

subset_data['mood'] = subset_data['cluster'].apply(assign_mood)


/usr/local/lib/python3.10/dist-packages/sklearn/cluster/_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(


**Explanation:**
Uses the K-means clustering algorithm to classify songs into different moods. The number of clusters (moods) is set to 5.

**Explanation:**
Defines a function to map each cluster (from K-means) to a mood label. This label is then added to the subset data.

In [ ]:
# 4. Interactive Widget Setup for the subset
subset_song_names = subset_data['name'].values
subset_song_features = subset_data[['valence', 'energy', 'danceability', 'acousticness', 'instrumentalness', 'liveness', 'loudness', 'speechiness', 'tempo']]
nn_model = NearestNeighbors(n_neighbors=50, metric='cosine', algorithm='brute')
nn_model.fit(subset_song_features)

song_selector = widgets.Dropdown(
    options=subset_song_names,
    description='Select Song:',
    disabled=False,
)

mood_selector = widgets.Dropdown(
    options=["Sad", "Neutral", "Happy/Energetic", "Relaxing", "Intense"],  # Replaced "Aggressive" with "Intense"
    description='Select Mood:',
    disabled=False,
)

button = widgets.Button(description="Recommend Songs")
output = widgets.Output()

def on_button_click(b):
    with output:
        clear_output()

        # Get selected song's features
        selected_song_features = subset_song_features[subset_data['name'] == song_selector.value].values

        # Collaborative Filtering
        distances, indices = nn_model.kneighbors(selected_song_features, n_neighbors=50)
        collab_recommended = subset_song_names[indices[0]]

        # Content-Based Filtering
        cosine_similarities = cosine_similarity(selected_song_features, subset_song_features)
        content_scores = cosine_similarities[0]
        content_indices = content_scores.argsort()[-50:][::-1]
        content_recommended = subset_song_names[content_indices]

        # Hybrid Recommendations
        combined_recommended = np.union1d(collab_recommended, content_recommended)

        # Mood Filtering
        selected_mood = mood_selector.value
        if selected_mood != "Neutral":
            final_recommendations = subset_data[subset_data['name'].isin(combined_recommended) & (subset_data['mood'] == selected_mood)]['name'].unique()
        else:
            final_recommendations = subset_data[subset_data['name'].isin(combined_recommended)]['name'].unique()

        # Display Recommendations
        print("Recommended Songs:")
        for song in final_recommendations:
            print(song)

button.on_click(on_button_click)

display(song_selector, mood_selector, button, output)

Dropdown(description='Select Song:', options=('Camby Bolongo', 'Just One of Those Things', 'Recognize', 'Soul …

Dropdown(description='Select Mood:', options=('Sad', 'Neutral', 'Happy/Energetic', 'Relaxing', 'Intense'), val…

Button(description='Recommend Songs', style=ButtonStyle())

Output()

**Explanation:**
The code creates dropdown menus for song selection and mood selection, and a button that, when clicked, recommends songs. The recommendations are based on a hybrid of collaborative and content-based filtering.